In [7]:
import os
from idlelib.rpc import response_queue

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI
from dotenv import load_dotenv

## Testing OpenAI API

In [10]:
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    raise ValueError('API key not found')
elif not api_key.startswith('sk-proj'):
    raise ValueError('Invalid API key')
elif api_key.strip() != api_key:
    raise ValueError('Invalid API key')
else:
    print('API key found')

API key found


In [13]:
openai = OpenAI()

message = 'Hello, OpenAI!'
response = openai.chat.completions.create(model='gpt-4o-mini', messages=[{'role': 'user', 'content': message}])
response = response.choices[0].message.content

In [14]:
response

'Hello! How can I assist you today?'

## First OpenAI API Project


In [33]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

class Website:
    def __init__(self, url):
        # Create this website object from the given URL using BeautifulSoup
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else 'No title found'
        for irrelevant in soup(['script', 'style', 'img', 'input']):
            irrelevant.decompose()
        self.text = soup.get_text()

In [35]:
# Test the website class

ed = Website('https://owliesol.com')
print(ed.title)
print(ed.text)

owliesol
owliesol


### Types of prompts
Models like GPT4o have been trained to receive instructions in a particular way.

They expect to receive:

A **system prompt** that tells them what task they are performing and what tone they should use

A **user prompt** -- the conversation starter that they should reply to

In [20]:
# Define the system prompt
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. Respond in markdown."

In [26]:
# Define the user prompt using a function

def user_promp_fort(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [27]:
# Test the prompts

user_promp_fort(ed)

'You are looking at a website titled owliesol\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nowliesol'

### Messages
The API from OpenAI expects to receive messages in a particular structure. Many of the other APIs share this structure:

```
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]
```

To give you a preview, the next 2 cells make a rather simple call - we won't stretch the might GPT (yet!)

In [28]:
messages = [
    {"role": "system", "content": "You are a snarky assistant"},
    {"role": "user", "content": "What is 2 + 2?"}
]

In [29]:
# To give you a preview -- calling OpenAI with system and user messages:

response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
print(response.choices[0].message.content)

Oh, you’re really going for the hard-hitting questions, huh? Well, that would be 4. Shocking, I know!


In [31]:
### GPT-4o Mini using a function

def message_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_promp_fort(website)}
    ]

In [36]:
message_for(ed)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled owliesol\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nowliesol'}]

### Calling OpenAI with the website object

In [37]:
def summarize_website(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=message_for(website)
    )
    return response.choices[0].message.content

In [38]:
summarize_website('https://owliesol.com')

'# Summary of OwlieSol\n\nThe OwlieSol website appears to focus on providing insights, resources, and tools related to a specific area of interest, possibly in technology or personal development. While detailed content specifics were not provided, the name suggests a brand possibly involved in learning or innovation.\n\n### Notable Features:\n- **Resources**: The site may offer a variety of helpful materials or guides.\n- **Community Engagement**: There may be opportunities for interaction among users or followers.\n\nSince no news or announcements were specified, there are currently no updates to report. \n\nFor a comprehensive understanding, visiting the site directly is recommended to explore its offerings and any recent updates.'

In [39]:
# A function to display output in

def display_summary(url):
    summary = summarize_website(url)
    display(Markdown(summary))

In [40]:
display_summary('https://owliesol.com')

# Summary of OwlieSol

OwlieSol is a website dedicated to providing insights, tools, and resources related to personal and professional development. It emphasizes the importance of lifelong learning and self-improvement, offering various articles, guides, and workshops designed to empower individuals in reaching their goals.

### Key Features:
- **Personal Development Resources:** The site offers various articles and materials focusing on self-improvement techniques and strategies.
- **Workshops and Courses:** Users can access information about upcoming workshops aimed at enhancing skills and knowledge.
- **Community Engagement:** The platform promotes a community-oriented approach, encouraging users to connect and share experiences.

### News/Announcements:
- **Upcoming Workshop:** There is an announcement regarding an upcoming workshop on effective communication skills, scheduled for next month.
- **New Blog Series Launch:** The website recently launched a new blog series focusing on mental wellness and its impact on productivity.

Overall, OwlieSol serves as a valuable resource for individuals seeking to enhance their personal and professional lives through education and community support.

In [41]:
display_summary("https://cnn.com")

# Summary of CNN's Breaking News Website

The **CNN** website serves as a hub for breaking news, featuring the latest updates across various domains, including US and world news, politics, business, health, and entertainment. It offers a variety of media formats, including videos and articles.

### Recent News Highlights:
- **Former President Bill Clinton** has been discharged from the hospital after flu treatment.
- **Russian cargo ship** sinks in the Mediterranean; 11 are reported dead following an explosion at an explosives factory in Turkey.
- **American Airlines** has resumed flights after resolving a technical issue that caused a nationwide groundstop.
- **Kilauea volcano** has started erupting in Hawaii, attracting holiday crowds.
- **Global Politics**: Trump is contemplating a territorial expansion involving Panama, Greenland, and Canada.

### Additional Notable Stories:
- **Social Dynamics**: Christians protested in Syria after the burning of a Christmas tree.
- A potential **massive volcanic eruption** is on the horizon, which experts fear could create widespread chaos.

Overall, CNN delivers timely news updates, drawing attention to critical issues and ongoing global events.

In [42]:
display_summary("https://anthropic.com")

# Summary of Anthropic Website

Anthropic is an AI safety and research company based in San Francisco, dedicated to creating reliable and beneficial AI systems. The website showcases its latest AI models, including Claude 3.5 Sonnet and Claude 3.5 Haiku, emphasizing their commitment to safety and alignment in AI development.

## Key Announcements:
- **Claude 3.5 Sonnet**: The latest intelligent AI model has been released, enhancing user interaction capabilities.
- **New Features**: A new feature for computer use was introduced alongside the 3.5 Sonnet model on October 22, 2024. 

## Research and Focus Areas:
Anthropic focuses on alignment research and the principles of AI safety, addressing crucial aspects like harmlessness through feedback and core views on AI safety.